In [1]:
import json

In [2]:
with open("C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018/"+'notes.json', 'r') as f:
    fname_to_notes = json.load(f)

In [3]:
fname = 'MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1.midi'

In [4]:
notes = fname_to_notes[fname]  # (pitch,start,end,velocity)

In [6]:
ss = set()
for n in notes:
    ss.add(n[0])
ss

{31,
 33,
 34,
 35,
 37,
 38,
 40,
 42,
 43,
 44,
 45,
 46,
 47,
 49,
 50,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 88,
 89}

In [9]:
def splitNotes(notes):
    new_notes = []
    for note in notes:
        start = note[1]/1000.
        end = note[2]/1000.
        startSec = int(start)
        endSec = int(end)

        if startSec == endSec:
            new_notes.append(note)
        else:
            new_notes.append([note[0],note[1],(startSec+1)*1000,note[3]])
            for i in range(startSec+1,endSec-1):
                new_notes.append([note[0],int(i*1000),int(i+1)*1000,note[3]])
            new_notes.append([note[0],int(endSec*1000),note[2],note[3]])
    return new_notes
            


In [10]:
updatedNotes = splitNotes(notes)

In [70]:
def getStrides(notes,size):
    notes = sorted(notes, key=lambda x: x[1])
    chunks = []
    for _ in range(len(notes)):
        chunks.append([])

    curT = 0
    nextidx = 0
    flag = False

    i = 0
    while(i<len(notes)):
        st = notes[i][1]
        maxT = curT+ size
        if st>=maxT:
            curT +=1000
            i = nextidx
            flag = False
            continue

        if st % 1000 == 0 and flag == False and st!=curT:
            nextidx = i
            flag = True
    
        pos = int(curT/1000)
        chunks[pos].append(notes[i])
        i+=1
    ans = []
    for c in chunks:
        if len(c)>0:
            ans.append(c)
    return ans


In [71]:
strides = getStrides(updatedNotes,5000)

In [83]:
len(strides[0])

51

In [81]:
import numpy as np

In [84]:
def getVector(notes):
    vec = np.zeros(128)
    pitch_to_drn = {}
    for note in notes:
        pitch = note[0]
        vel = note[3]
        drn = note[2]-note[1]
        if pitch in pitch_to_drn:
            pitch_to_drn[pitch] += drn
        else:
            pitch_to_drn[pitch] = drn
    
    for pitch in pitch_to_drn:
        vec[pitch] = pitch_to_drn[pitch]
    return vec

In [85]:
getVector(strides[0])

array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,  471.,    0.,    0.,    0.,  144., 2243.,
          0.,    0.,    0., 1659.,  122.,    0., 1098.,    0.,    0.,
          0.,    0.,  579.,    0.,  189.,    0., 1348.,    0.,    0.,
       1257.,    0.,    0.,    0.,    0.,  143.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.]

In [86]:
def getSim(queryVec, docVec):
    if np.linalg.norm(queryVec) == 0 or np.linalg.norm(docVec) == 0:
        return 0.0
    return np.dot(queryVec, docVec)/(np.linalg.norm(queryVec) * np.linalg.norm(docVec))

In [93]:
getSim(getVector(strides[0]),getVector(strides[4]))

0.9899886151082533

In [88]:
getSim(getVector(strides[1]), getVector(strides[2]))


0.9867138053699629

In [97]:
fn = "MIDI-Unprocessed_Chamber4_MID--AUDIO_11_R3_2018_wav--1.midi"
notes2 = fname_to_notes[fn]  # (pitch,start,end,velocity)
updatedNotes2 = splitNotes(notes2)
strides2 = getStrides(updatedNotes2, 5000)
getSim(getVector(strides[0]),getVector(strides2[11]))

0.4202470286393309